<a href="https://colab.research.google.com/github/Tmccoy22/fintech_project_2/blob/main/nfl_predictions_running_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Imports And Clean the Data

In [1]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn import svm
import tensorflow as tp
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
# Upload credit_card_transactions.csv to Colab
from google.colab import files

csv_file = files.upload()

# Read the applicants_data.csv file from the Resources folder into a Pandas DataFrame
NFL_df = pd.read_csv("NFL_data.csv")

# Review the DataFrame
NFL_df.head()

Saving NFL_data.csv to NFL_data (1).csv


,Date,Home Team,Away Team,Home Score,Away Score,Overtime?,Playoff Game?,Neutral Venue?,Home Odds Open,Home Odds Min,...,Total Score Close,Total Score Over Open,Total Score Over Min,Total Score Over Max,Total Score Over Close,Total Score Under Open,Total Score Under Min,Total Score Under Max,Total Score Under Close,Notes
0,2023-02-12,Philadelphia Eagles,Kansas City Chiefs,35,38,NaN,Y,Y,1.83,1.74,...,51.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,NaN
1,2023-01-29,Kansas City Chiefs,Cincinnati Bengals,23,20,NaN,Y,NaN,1.87,1.77,...,48.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,NaN
2,2023-01-29,Philadelphia Eagles,San Francisco 49ers,31,7,NaN,Y,NaN,1.77,1.65,...,45.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,NaN
3,2023-01-22,San Francisco 49ers,Dallas Cowboys,19,12,NaN,Y,NaN,1.50,1.48,...,46.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,NaN
4,2023-01-22,Buffalo Bills,Cincinnati Bengals,10,27,NaN,Y,NaN,1.52,1.38,...,48.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,NaN


In [3]:
#Drop Unused columns
NFL_odds_df = NFL_df.drop(columns = ["Overtime?", "Playoff Game?", "Neutral Venue?", "Notes", "Total Score Open", "Total Score Min", "Total Score Close",
               "Total Score Over Open","Total Score Over Min", "Total Score Over Max", "Total Score Over Close", "Total Score Under Open", 
               "Total Score Under Min" , "Total Score Under Max", "Total Score Under Close", "Total Score Max", "Date"])

In [4]:
NFL_odds_df.dtypes

Home Team                object
Away Team                object
Home Score                int64
Away Score                int64
Home Odds Open          float64
Home Odds Min           float64
Home Odds Max           float64
Home Odds Close         float64
Away Odds Open          float64
Away Odds Min           float64
Away Odds Max           float64
Away Odds Close         float64
Home Line Open          float64
Home Line Min           float64
Home Line Max           float64
Home Line Close         float64
Away Line Open          float64
Away Line Min           float64
Away Line Max           float64
Away Line Close         float64
Home Line Odds Open     float64
Home Line Odds Min      float64
Home Line Odds Max      float64
Home Line Odds Close    float64
Away Line Odds Open     float64
Away Line Odds Min      float64
Away Line Odds Max      float64
Away Line Odds Close    float64
dtype: object

In [5]:
#NFL_odds_df = NFL_odds_df.dropna()

In [6]:
NFL_odds_df['Home Team'] = NFL_odds_df['Home Team'].fillna(int(0), inplace= True)

In [7]:
NFL_odds_df['Away Team'] = NFL_odds_df['Away Team'].fillna(int(1), inplace = True)

In [8]:
NFL_odds_df['Home Team'] = NFL_odds_df['Home Team'].fillna(int(0))

In [9]:
NFL_odds_df['Away Team'] = NFL_odds_df['Away Team'].fillna(int(1),)

In [10]:
NFL_odds_df.head()

,Home Team,Away Team,Home Score,Away Score,Home Odds Open,Home Odds Min,Home Odds Max,Home Odds Close,Away Odds Open,Away Odds Min,...,Away Line Max,Away Line Close,Home Line Odds Open,Home Line Odds Min,Home Line Odds Max,Home Line Odds Close,Away Line Odds Open,Away Line Odds Min,Away Line Odds Max,Away Line Odds Close
0,0,1,35,38,1.83,1.74,1.83,1.83,2.00,2.00,...,2.5,1.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91
1,0,1,23,20,1.87,1.77,2.20,1.77,1.95,1.71,...,2.0,2.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91
2,0,1,31,7,1.77,1.65,1.77,1.65,2.10,2.10,...,3.0,3.0,1.87,2.00,1.87,2.00,1.95,1.95,1.83,1.83
3,0,1,19,12,1.50,1.48,1.54,1.50,2.70,2.65,...,4.0,3.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91
4,0,1,10,27,1.52,1.38,1.52,1.38,2.67,2.67,...,6.0,6.0,1.95,1.91,1.95,1.91,1.87,1.87,1.91,1.91


In [11]:
#winner = []
#for i, row in NFL_odds_df.iterrows():
    #if row['Home Score'] > row['Away Score']:
     #   winner.append(row['Home Team'])
    #elif row['Away Score'] > row['Home Score']:
     #   winner.append(row['Away Team'])
    # Add the following line to skip appending a team name for tie games
    # and only append team names for games with a clear winner:
    #elif row['Home Score'] == row['Away Score']:
     #   continue

In [12]:
winner = []
for i, row in NFL_odds_df.iterrows():
    if row['Home Score'] > row['Away Score']:
        winner.append(row['Home Team'])
    elif row['Away Score'] > row['Home Score']:
        winner.append(row['Away Team'])
    else:
        winner.append(pd.np.nan)

<ipython-input-12-6bd6564e0bc0>:8: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  winner.append(pd.np.nan)


In [13]:
NFL_odds_df['winner'] = winner

In [14]:
NFL_odds_df.head()

,Home Team,Away Team,Home Score,Away Score,Home Odds Open,Home Odds Min,Home Odds Max,Home Odds Close,Away Odds Open,Away Odds Min,...,Away Line Close,Home Line Odds Open,Home Line Odds Min,Home Line Odds Max,Home Line Odds Close,Away Line Odds Open,Away Line Odds Min,Away Line Odds Max,Away Line Odds Close,winner
0,0,1,35,38,1.83,1.74,1.83,1.83,2.00,2.00,...,1.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.0
1,0,1,23,20,1.87,1.77,2.20,1.77,1.95,1.71,...,2.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,0.0
2,0,1,31,7,1.77,1.65,1.77,1.65,2.10,2.10,...,3.0,1.87,2.00,1.87,2.00,1.95,1.95,1.83,1.83,0.0
3,0,1,19,12,1.50,1.48,1.54,1.50,2.70,2.65,...,3.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,0.0
4,0,1,10,27,1.52,1.38,1.52,1.38,2.67,2.67,...,6.0,1.95,1.91,1.95,1.91,1.87,1.87,1.91,1.91,1.0


In [15]:
NFL_odds_df = NFL_odds_df.dropna()

In [16]:
# NFL_odds_filtered = NFL_odds_df[NFL_odds_df["winner"]!='Tie']

In [17]:
#NFL_odds_filtered

In [18]:
NFL_odds_df

,Home Team,Away Team,Home Score,Away Score,Home Odds Open,Home Odds Min,Home Odds Max,Home Odds Close,Away Odds Open,Away Odds Min,...,Away Line Close,Home Line Odds Open,Home Line Odds Min,Home Line Odds Max,Home Line Odds Close,Away Line Odds Open,Away Line Odds Min,Away Line Odds Max,Away Line Odds Close,winner
0,0,1,35,38,1.83,1.74,1.83,1.83,2.00,2.00,...,1.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.0
1,0,1,23,20,1.87,1.77,2.20,1.77,1.95,1.71,...,2.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,0.0
2,0,1,31,7,1.77,1.65,1.77,1.65,2.10,2.10,...,3.0,1.87,2.00,1.87,2.00,1.95,1.95,1.83,1.83,0.0
3,0,1,19,12,1.50,1.48,1.54,1.50,2.70,2.65,...,3.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,0.0
4,0,1,10,27,1.52,1.38,1.52,1.38,2.67,2.67,...,6.0,1.95,1.91,1.95,1.91,1.87,1.87,1.91,1.91,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2435,0,1,19,14,1.43,1.34,1.43,1.36,3.10,3.05,...,6.5,1.95,1.91,1.97,1.93,1.95,1.93,2.00,1.98,0.0
2436,0,1,34,17,1.20,1.18,1.21,1.19,5.23,5.08,...,10.0,1.91,1.93,2.02,2.02,2.00,1.88,1.98,1.88,0.0
2437,0,1,30,27,1.37,1.32,1.42,1.41,3.39,3.14,...,5.5,1.92,1.97,1.95,1.93,1.99,1.95,1.93,1.97,0.0
2438,0,1,6,34,1.50,1.50,1.73,1.65,2.81,2.24,...,3.0,1.95,1.95,1.88,2.02,1.95,2.02,1.95,1.88,1.0


In [19]:
NFL_odds_df.dropna()

,Home Team,Away Team,Home Score,Away Score,Home Odds Open,Home Odds Min,Home Odds Max,Home Odds Close,Away Odds Open,Away Odds Min,...,Away Line Close,Home Line Odds Open,Home Line Odds Min,Home Line Odds Max,Home Line Odds Close,Away Line Odds Open,Away Line Odds Min,Away Line Odds Max,Away Line Odds Close,winner
0,0,1,35,38,1.83,1.74,1.83,1.83,2.00,2.00,...,1.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.0
1,0,1,23,20,1.87,1.77,2.20,1.77,1.95,1.71,...,2.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,0.0
2,0,1,31,7,1.77,1.65,1.77,1.65,2.10,2.10,...,3.0,1.87,2.00,1.87,2.00,1.95,1.95,1.83,1.83,0.0
3,0,1,19,12,1.50,1.48,1.54,1.50,2.70,2.65,...,3.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,0.0
4,0,1,10,27,1.52,1.38,1.52,1.38,2.67,2.67,...,6.0,1.95,1.91,1.95,1.91,1.87,1.87,1.91,1.91,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2435,0,1,19,14,1.43,1.34,1.43,1.36,3.10,3.05,...,6.5,1.95,1.91,1.97,1.93,1.95,1.93,2.00,1.98,0.0
2436,0,1,34,17,1.20,1.18,1.21,1.19,5.23,5.08,...,10.0,1.91,1.93,2.02,2.02,2.00,1.88,1.98,1.88,0.0
2437,0,1,30,27,1.37,1.32,1.42,1.41,3.39,3.14,...,5.5,1.92,1.97,1.95,1.93,1.99,1.95,1.93,1.97,0.0
2438,0,1,6,34,1.50,1.50,1.73,1.65,2.81,2.24,...,3.0,1.95,1.95,1.88,2.02,1.95,2.02,1.95,1.88,1.0


In [20]:
data_df = NFL_odds_df.drop(columns=['Home Score', 'Away Score'])

In [21]:
data_df.head()

,Home Team,Away Team,Home Odds Open,Home Odds Min,Home Odds Max,Home Odds Close,Away Odds Open,Away Odds Min,Away Odds Max,Away Odds Close,...,Away Line Close,Home Line Odds Open,Home Line Odds Min,Home Line Odds Max,Home Line Odds Close,Away Line Odds Open,Away Line Odds Min,Away Line Odds Max,Away Line Odds Close,winner
0,0,1,1.83,1.74,1.83,1.83,2.00,2.00,2.15,2.00,...,1.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.0
1,0,1,1.87,1.77,2.20,1.77,1.95,1.71,2.10,2.10,...,2.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,0.0
2,0,1,1.77,1.65,1.77,1.65,2.10,2.10,2.35,2.35,...,3.0,1.87,2.00,1.87,2.00,1.95,1.95,1.83,1.83,0.0
3,0,1,1.50,1.48,1.54,1.50,2.70,2.65,2.75,2.70,...,3.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,0.0
4,0,1,1.52,1.38,1.52,1.38,2.67,2.67,3.15,3.15,...,6.0,1.95,1.91,1.95,1.91,1.87,1.87,1.91,1.91,1.0


##Estabolish the Nurral Network

In [22]:
y = data_df['winner']

In [23]:
y.head()

0    1.0
1    0.0
2    0.0
3    0.0
4    1.0
Name: winner, dtype: float64

In [24]:
X = data_df.drop(columns='winner')

In [25]:
X.head()

,Home Team,Away Team,Home Odds Open,Home Odds Min,Home Odds Max,Home Odds Close,Away Odds Open,Away Odds Min,Away Odds Max,Away Odds Close,...,Away Line Max,Away Line Close,Home Line Odds Open,Home Line Odds Min,Home Line Odds Max,Home Line Odds Close,Away Line Odds Open,Away Line Odds Min,Away Line Odds Max,Away Line Odds Close
0,0,1,1.83,1.74,1.83,1.83,2.00,2.00,2.15,2.00,...,2.5,1.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91
1,0,1,1.87,1.77,2.20,1.77,1.95,1.71,2.10,2.10,...,2.0,2.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91
2,0,1,1.77,1.65,1.77,1.65,2.10,2.10,2.35,2.35,...,3.0,3.0,1.87,2.00,1.87,2.00,1.95,1.95,1.83,1.83
3,0,1,1.50,1.48,1.54,1.50,2.70,2.65,2.75,2.70,...,4.0,3.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91
4,0,1,1.52,1.38,1.52,1.38,2.67,2.67,3.15,3.15,...,6.0,6.0,1.95,1.91,1.95,1.91,1.87,1.87,1.91,1.91


In [26]:
X.tail()

,Home Team,Away Team,Home Odds Open,Home Odds Min,Home Odds Max,Home Odds Close,Away Odds Open,Away Odds Min,Away Odds Max,Away Odds Close,...,Away Line Max,Away Line Close,Home Line Odds Open,Home Line Odds Min,Home Line Odds Max,Home Line Odds Close,Away Line Odds Open,Away Line Odds Min,Away Line Odds Max,Away Line Odds Close
2435,0,1,1.43,1.34,1.43,1.36,3.10,3.05,3.56,3.46,...,6.5,6.5,1.95,1.91,1.97,1.93,1.95,1.93,2.00,1.98
2436,0,1,1.20,1.18,1.21,1.19,5.23,5.08,5.54,5.35,...,10.5,10.0,1.91,1.93,2.02,2.02,2.00,1.88,1.98,1.88
2437,0,1,1.37,1.32,1.42,1.41,3.39,3.14,3.73,3.16,...,7.0,5.5,1.92,1.97,1.95,1.93,1.99,1.95,1.93,1.97
2438,0,1,1.50,1.50,1.73,1.65,2.81,2.24,2.81,2.40,...,4.0,3.0,1.95,1.95,1.88,2.02,1.95,2.02,1.95,1.88
2439,0,1,1.45,1.38,1.47,1.43,2.99,2.91,3.32,3.05,...,6.0,4.5,1.97,1.93,2.00,1.93,1.93,1.91,1.98,1.98


In [27]:
print(X.isna().any().any())

False


In [28]:
print(y.isna().any().any())

False


In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [30]:
scaler = StandardScaler()

In [31]:
X_scaler = scaler.fit(X_train)

In [32]:
# Do we need to drop home/away scores? This is obviously the strongest predictor of who wins. Will model just focus on this?
# TensorFlow doesnt work in Jupyter lab?

In [33]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [34]:
#X_train= np.asarray(X_train).astype(np.float32)
#y_train = np.asarray(y_train).astype(np.float32)

In [35]:
#y_train = np.array(y_train)

In [36]:
number_input_features = len(X_train.iloc[0])

In [37]:
number_input_features

26

In [38]:
number_output_neurons = (1)

In [39]:
hidden_nodes_layer1 =  (number_input_features + 1) // 2 

In [40]:
hidden_nodes_layer1

13

In [41]:
hidden_nodes_layer2 =  (hidden_nodes_layer1 + 1) // 2

In [42]:
hidden_nodes_layer2

7

In [43]:
nn = Sequential()

In [44]:
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

In [45]:
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

In [46]:
nn.add(Dense(units=number_output_neurons, activation="sigmoid"))

In [47]:
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 13)                351       
                                                                 
 dense_1 (Dense)             (None, 7)                 98        
                                                                 
 dense_2 (Dense)             (None, 1)                 8         
                                                                 
Total params: 457
Trainable params: 457
Non-trainable params: 0
_________________________________________________________________


In [48]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [49]:
nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
57/57 [==============================] - 2s 2ms/step - loss: 0.6552 - accuracy: 0.6081
Epoch 2/100
57/57 [==============================] - 0s 2ms/step - loss: 0.6178 - accuracy: 0.6531
Epoch 3/100
57/57 [==============================] - 0s 3ms/step - loss: 0.6129 - accuracy: 0.6597
Epoch 4/100
57/57 [==============================] - 0s 3ms/step - loss: 0.6121 - accuracy: 0.6614
Epoch 5/100
57/57 [==============================] - 0s 2ms/step - loss: 0.6108 - accuracy: 0.6663
Epoch 6/100
57/57 [==============================] - 0s 3ms/step - loss: 0.6101 - accuracy: 0.6608
Epoch 7/100
57/57 [==============================] - 0s 2ms/step - loss: 0.6092 - accuracy: 0.6614
Epoch 8/100
57/57 [==============================] - 0s 2ms/step - loss: 0.6083 - accuracy: 0.6603
Epoch 9/100
57/57 [==============================] - 0s 2ms/step - loss: 0.6078 - accuracy: 0.6597
Epoch 10/100
57/57 [==============================] - 0s 2ms/step - loss: 0.6071 - accuracy: 0.6658
Epoch 11/

Model 1 results

In [50]:

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

19/19 - 0s - loss: 0.6351 - accuracy: 0.6316 - 195ms/epoch - 10ms/step
Loss: 0.6350579261779785, Accuracy: 0.6315789222717285


##Alternate Model 

In [51]:
y = data_df['winner']

In [52]:
y.head()

0    1.0
1    0.0
2    0.0
3    0.0
4    1.0
Name: winner, dtype: float64

In [53]:
X = data_df.drop(columns='winner')

In [54]:
X.head()

,Home Team,Away Team,Home Odds Open,Home Odds Min,Home Odds Max,Home Odds Close,Away Odds Open,Away Odds Min,Away Odds Max,Away Odds Close,...,Away Line Max,Away Line Close,Home Line Odds Open,Home Line Odds Min,Home Line Odds Max,Home Line Odds Close,Away Line Odds Open,Away Line Odds Min,Away Line Odds Max,Away Line Odds Close
0,0,1,1.83,1.74,1.83,1.83,2.00,2.00,2.15,2.00,...,2.5,1.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91
1,0,1,1.87,1.77,2.20,1.77,1.95,1.71,2.10,2.10,...,2.0,2.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91
2,0,1,1.77,1.65,1.77,1.65,2.10,2.10,2.35,2.35,...,3.0,3.0,1.87,2.00,1.87,2.00,1.95,1.95,1.83,1.83
3,0,1,1.50,1.48,1.54,1.50,2.70,2.65,2.75,2.70,...,4.0,3.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91
4,0,1,1.52,1.38,1.52,1.38,2.67,2.67,3.15,3.15,...,6.0,6.0,1.95,1.91,1.95,1.91,1.87,1.87,1.91,1.91


In [55]:
X.tail()

,Home Team,Away Team,Home Odds Open,Home Odds Min,Home Odds Max,Home Odds Close,Away Odds Open,Away Odds Min,Away Odds Max,Away Odds Close,...,Away Line Max,Away Line Close,Home Line Odds Open,Home Line Odds Min,Home Line Odds Max,Home Line Odds Close,Away Line Odds Open,Away Line Odds Min,Away Line Odds Max,Away Line Odds Close
2435,0,1,1.43,1.34,1.43,1.36,3.10,3.05,3.56,3.46,...,6.5,6.5,1.95,1.91,1.97,1.93,1.95,1.93,2.00,1.98
2436,0,1,1.20,1.18,1.21,1.19,5.23,5.08,5.54,5.35,...,10.5,10.0,1.91,1.93,2.02,2.02,2.00,1.88,1.98,1.88
2437,0,1,1.37,1.32,1.42,1.41,3.39,3.14,3.73,3.16,...,7.0,5.5,1.92,1.97,1.95,1.93,1.99,1.95,1.93,1.97
2438,0,1,1.50,1.50,1.73,1.65,2.81,2.24,2.81,2.40,...,4.0,3.0,1.95,1.95,1.88,2.02,1.95,2.02,1.95,1.88
2439,0,1,1.45,1.38,1.47,1.43,2.99,2.91,3.32,3.05,...,6.0,4.5,1.97,1.93,2.00,1.93,1.93,1.91,1.98,1.98


In [56]:
print(X.isna().any().any())

False


In [57]:
print(y.isna().any().any())

False


In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [59]:
scaler = StandardScaler()

In [60]:
X_scaler = scaler.fit(X_train)

In [61]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [62]:
number_input_features = len(X_train.iloc[0])

In [63]:
number_input_features

26

In [64]:
number_output_neurons = (1)

In [65]:
hidden_nodes_layer1 =  (number_input_features + 1) // 2 

In [66]:
hidden_nodes_layer1

13

In [67]:
hidden_nodes_layer2 =  (hidden_nodes_layer1 + 1) // 2

In [68]:
hidden_nodes_layer2

7

In [69]:
hidden_nodes_layer3 = (hidden_nodes_layer2+1)//2

In [70]:
hidden_nodes_layer3

4

In [71]:
nn2 = Sequential()

In [72]:
nn2.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

In [73]:
nn2.add(Dense(units=hidden_nodes_layer2, activation="relu"))

In [74]:
nn2.add(Dense(units = hidden_nodes_layer3, activation = 'relu'))

In [75]:
nn2.add(Dense(units=number_output_neurons, activation="sigmoid"))

In [76]:
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 13)                351       
                                                                 
 dense_4 (Dense)             (None, 7)                 98        
                                                                 
 dense_5 (Dense)             (None, 4)                 32        
                                                                 
 dense_6 (Dense)             (None, 1)                 5         
                                                                 
Total params: 486
Trainable params: 486
Non-trainable params: 0
_________________________________________________________________


In [77]:
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

### increase epochs and added extra layer

In [78]:
nn2.fit(X_train_scaled, y_train, epochs=125)

Epoch 1/125
57/57 [==============================] - 1s 2ms/step - loss: 0.6887 - accuracy: 0.5055
Epoch 2/125
57/57 [==============================] - 0s 2ms/step - loss: 0.6473 - accuracy: 0.5499
Epoch 3/125
57/57 [==============================] - 0s 2ms/step - loss: 0.6390 - accuracy: 0.5867
Epoch 4/125
57/57 [==============================] - 0s 2ms/step - loss: 0.6347 - accuracy: 0.6575
Epoch 5/125
57/57 [==============================] - 0s 2ms/step - loss: 0.6323 - accuracy: 0.6641
Epoch 6/125
57/57 [==============================] - 0s 2ms/step - loss: 0.6297 - accuracy: 0.6679
Epoch 7/125
57/57 [==============================] - 0s 2ms/step - loss: 0.6276 - accuracy: 0.6668
Epoch 8/125
57/57 [==============================] - 0s 2ms/step - loss: 0.6259 - accuracy: 0.6701
Epoch 9/125
57/57 [==============================] - 0s 2ms/step - loss: 0.6246 - accuracy: 0.6690
Epoch 10/125
57/57 [==============================] - 0s 2ms/step - loss: 0.6245 - accuracy: 0.6674
Epoch 11/

Model 2 Results

In [79]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

19/19 - 0s - loss: 0.6527 - accuracy: 0.6266 - 234ms/epoch - 12ms/step
Loss: 0.652715802192688, Accuracy: 0.6266447305679321


##Alternate Model 2


In [80]:
y = data_df['winner']

In [81]:
y.head()

0    1.0
1    0.0
2    0.0
3    0.0
4    1.0
Name: winner, dtype: float64

In [82]:
X = data_df.drop(columns='winner')

In [83]:
X.head()

,Home Team,Away Team,Home Odds Open,Home Odds Min,Home Odds Max,Home Odds Close,Away Odds Open,Away Odds Min,Away Odds Max,Away Odds Close,...,Away Line Max,Away Line Close,Home Line Odds Open,Home Line Odds Min,Home Line Odds Max,Home Line Odds Close,Away Line Odds Open,Away Line Odds Min,Away Line Odds Max,Away Line Odds Close
0,0,1,1.83,1.74,1.83,1.83,2.00,2.00,2.15,2.00,...,2.5,1.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91
1,0,1,1.87,1.77,2.20,1.77,1.95,1.71,2.10,2.10,...,2.0,2.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91
2,0,1,1.77,1.65,1.77,1.65,2.10,2.10,2.35,2.35,...,3.0,3.0,1.87,2.00,1.87,2.00,1.95,1.95,1.83,1.83
3,0,1,1.50,1.48,1.54,1.50,2.70,2.65,2.75,2.70,...,4.0,3.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91
4,0,1,1.52,1.38,1.52,1.38,2.67,2.67,3.15,3.15,...,6.0,6.0,1.95,1.91,1.95,1.91,1.87,1.87,1.91,1.91


In [84]:
X.tail()

,Home Team,Away Team,Home Odds Open,Home Odds Min,Home Odds Max,Home Odds Close,Away Odds Open,Away Odds Min,Away Odds Max,Away Odds Close,...,Away Line Max,Away Line Close,Home Line Odds Open,Home Line Odds Min,Home Line Odds Max,Home Line Odds Close,Away Line Odds Open,Away Line Odds Min,Away Line Odds Max,Away Line Odds Close
2435,0,1,1.43,1.34,1.43,1.36,3.10,3.05,3.56,3.46,...,6.5,6.5,1.95,1.91,1.97,1.93,1.95,1.93,2.00,1.98
2436,0,1,1.20,1.18,1.21,1.19,5.23,5.08,5.54,5.35,...,10.5,10.0,1.91,1.93,2.02,2.02,2.00,1.88,1.98,1.88
2437,0,1,1.37,1.32,1.42,1.41,3.39,3.14,3.73,3.16,...,7.0,5.5,1.92,1.97,1.95,1.93,1.99,1.95,1.93,1.97
2438,0,1,1.50,1.50,1.73,1.65,2.81,2.24,2.81,2.40,...,4.0,3.0,1.95,1.95,1.88,2.02,1.95,2.02,1.95,1.88
2439,0,1,1.45,1.38,1.47,1.43,2.99,2.91,3.32,3.05,...,6.0,4.5,1.97,1.93,2.00,1.93,1.93,1.91,1.98,1.98


In [85]:
print(X.isna().any().any())

False


In [86]:
print(y.isna().any().any())

False


In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [88]:
scaler = StandardScaler()

In [89]:
X_scaler = scaler.fit(X_train)

In [90]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [91]:
number_input_features = len(X_train.iloc[0])

In [92]:
number_input_features

26

In [93]:
number_output_neurons = (1)

In [94]:
hidden_nodes_layer1 =  (number_input_features + 1) // 2 

In [95]:
hidden_nodes_layer1

13

In [96]:
hidden_nodes_layer2 =  (hidden_nodes_layer1 + 1) // 2

In [97]:
hidden_nodes_layer2

7

In [98]:
nn3 = Sequential()

In [99]:
nn3.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

In [100]:
nn3.add(Dense(units=hidden_nodes_layer2, activation="relu"))

In [101]:
nn3.add(Dense(units=number_output_neurons, activation="sigmoid"))

In [102]:
nn3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 13)                351       
                                                                 
 dense_8 (Dense)             (None, 7)                 98        
                                                                 
 dense_9 (Dense)             (None, 1)                 8         
                                                                 
Total params: 457
Trainable params: 457
Non-trainable params: 0
_________________________________________________________________


In [103]:
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [104]:
nn3.fit(X_train_scaled, y_train, epochs=500)

Epoch 1/500
57/57 [==============================] - 1s 2ms/step - loss: 0.6860 - accuracy: 0.5291
Epoch 2/500
57/57 [==============================] - 0s 2ms/step - loss: 0.6328 - accuracy: 0.6542
Epoch 3/500
57/57 [==============================] - 0s 2ms/step - loss: 0.6148 - accuracy: 0.6630
Epoch 4/500
57/57 [==============================] - 0s 2ms/step - loss: 0.6133 - accuracy: 0.6652
Epoch 5/500
57/57 [==============================] - 0s 2ms/step - loss: 0.6120 - accuracy: 0.6663
Epoch 6/500
57/57 [==============================] - 0s 2ms/step - loss: 0.6107 - accuracy: 0.6641
Epoch 7/500
57/57 [==============================] - 0s 2ms/step - loss: 0.6088 - accuracy: 0.6636
Epoch 8/500
57/57 [==============================] - 0s 2ms/step - loss: 0.6083 - accuracy: 0.6614
Epoch 9/500
57/57 [==============================] - 0s 2ms/step - loss: 0.6071 - accuracy: 0.6614
Epoch 10/500
57/57 [==============================] - 0s 2ms/step - loss: 0.6064 - accuracy: 0.6597
Epoch 11/

In [105]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss3, model_accuracy3 = nn3.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss3}, Accuracy: {model_accuracy3}")

19/19 - 0s - loss: 0.6735 - accuracy: 0.6382 - 152ms/epoch - 8ms/step
Loss: 0.6735411882400513, Accuracy: 0.6381579041481018
